In [58]:
import os
import re
import json
import boto3
import requests
from dotenv import load_dotenv

load_dotenv()
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY')
REGION_NAME = os.environ.get('REGION_NAME')

dynamodb = boto3.resource('dynamodb', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY, region_name=REGION_NAME)

access_token = 'BQDuCQferFcFbzB2pOQ_vH_39oR3XYU-cVty6JfSNFFnF5iEK1hRvUrut1qyuteTIU7eFCFQGz6K9YseH2ajgxOqUzO_qIaabOOo4ktfYuYI9AodJ6-VGyIixwBwhx1SJCQk_LqiU_oyNq8yQ2AO-ibTYe_XS-PG4FxeKd3i-mPwufsiEebPbUvmPzUekTREDAxUxp454R0UU0S_PFyWzQ'

# Helper Functions

In [65]:
def get_playlist_uri(playlist_url: str):
    return re.search(r'/([^/]+)\?', playlist_url).group(1)

def get_track_ids_and_store_audio_features(access_token: str, playlist_uri: str):
    playlist_info = {
        'track_ids': [],
        'playlist_uri': playlist_uri,
    }

    tracks_table = dynamodb.Table('SPOTIFY_TRACKS')

    headers = {
        'Authorization': f"Bearer {access_token}"
    }

    print(f"fetching playlist {playlist_uri}")
    raw_json_to_check = {'next': f"https://api.spotify.com/v1/playlists/{playlist_uri}/tracks?market=US&limit=50"}

    num = 1
    while raw_json_to_check['next'] != None:
        print(f"batch {num}")
        response = requests.get(url = raw_json_to_check['next'], headers = headers)

        if response.status_code == 200:
            playlist_raw_json = response.json()

            map_track_ids = []
            for item in playlist_raw_json['items']:
                if item['track'] != None and item['track']['id'] not in playlist_info['track_ids']:
                    track_id = item['track']['id']

                    if track_id != None:
                        map_track_ids.append({'id': track_id})
                    playlist_info['track_ids'].append(track_id)
            
            track_ids = []
            index = 0
            while index < len(map_track_ids) - 1:
                db_response = dynamodb.batch_get_item(
                    RequestItems={
                        'SPOTIFY_TRACKS': {
                            'Keys': map_track_ids[index:index + 100]
                        }
                    }
                )
                track_received = db_response['Responses']['SPOTIFY_TRACKS']
                track_ids_received = {}
                for track in track_received:
                    track_ids_received[track['id']] = 1

                # add track if havent been added
                max_value = index + 100
                if max_value > len(map_track_ids):
                    max_value = len(map_track_ids)
                
                for i in range(index, max_value):
                    track_id = map_track_ids[i]['id']

                    if track_id not in track_ids_received:
                        index += 1
                        track_ids.append(track_id)
                
                index += len(track_received) - 1

            if len(track_ids) != 0:
                query = {
                    'ids': ','.join(track_ids)
                }
                response = requests.get(url = 'https://api.spotify.com/v1/audio-features', headers = headers, params = query)

                if response.status_code == 200:
                    raw_json = response.json()
                    
                    i = 0
                    while i < len(raw_json['audio_features']):
                        audio_features = raw_json['audio_features'][i]

                        if audio_features != "None" and audio_features != None:
                            for key, value in audio_features.items():
                                if isinstance(value, float):
                                    raw_json['audio_features'][i][key] = str(value)
                                    
                        else:
                            raw_json['audio_features'].pop(i)
                            i -= 1

                        i += 1
                    
                    for track in raw_json['audio_features']:
                        tracks_table.put_item(Item=track)

                else:
                    print(response.text)

            num += 1
            raw_json_to_check = playlist_raw_json

        else:
            print(response.text)
    
    return playlist_info

# Initial Zodiac Playlist Scraper

In [262]:
playlists = [
    ['https://open.spotify.com/playlist/7BIFvKEvSWfr0c1Ur2PxQ5?si=34101201091d4f3f', 'https://open.spotify.com/playlist/6w2nktImGPS83JiAkG2VzB?si=c6539d4ff9f942bf', 'https://open.spotify.com/playlist/5NZj30WhMY4gnMIGPD8gir?si=24cc7fa618754012'],
    ['https://open.spotify.com/playlist/7LDJsFK7uk53RNxax3z2YC?si=efa7f1f9e09a4268', 'https://open.spotify.com/playlist/0eN9zu6ZezHsNodVtHIlAU?si=34d8625b252e4c45', 'https://open.spotify.com/playlist/6unGtxWr0zGP2GJQijrzTW?si=253b268341974cce'],
    ['https://open.spotify.com/playlist/3kTgfkfU6GomvrEyNgC7LV?si=8490e9c85b8242ff', 'https://open.spotify.com/playlist/6SP3BZKRwpp5PX0rI1Rmem?si=d121334278574748', 'https://open.spotify.com/playlist/1AmBxyen2AaSKf6xgl4FAa?si=0e29e78eb1da4fff'],
    ['https://open.spotify.com/playlist/6zut6b4UaLmsxVPvTHAyUC?si=f1347e12a7e34a0d', 'https://open.spotify.com/playlist/3n5J91l343pfMflirtBZhD?si=0765cda31d53453f', 'https://open.spotify.com/playlist/5Wlx45esaqtfimuHPGYaF4?si=a86f7f3b46d346c7', 'https://open.spotify.com/playlist/003iGZIa5Ne43VBOjxe8OQ?'],
    ['https://open.spotify.com/playlist/3GzH2VIpDXQaouhWJeBvYn?si=d2f3001e9a754ca4', 'https://open.spotify.com/playlist/4OUEfwVUeRHzAKW5rGjH3M?si=0c22d35408734834', 'https://open.spotify.com/playlist/6XmxwdTuFrNQJjhUOEPki3?si=476b15ffd94a4995', 'https://open.spotify.com/playlist/3QHU58OJaPRKQY8oLh7vbz?', 'https://open.spotify.com/playlist/6OOrrtVCeyjUQNeh5WCeUq?', 'https://open.spotify.com/playlist/6QUK0oaKo6hxOVO3d39r4n?'],
    ['https://open.spotify.com/playlist/3thnEzH8MTVm5t4kyyNVGV?si=755d65300bc64b75', 'https://open.spotify.com/playlist/2k2n89VO9E6uYsxPvatLsk?si=9bfe933f2345489a', 'https://open.spotify.com/playlist/4ytPJrzShCo3iftbTgHi4p?si=8bbde85ad4c44980'],
    ['https://open.spotify.com/playlist/0c9nCMbp3TcR8aLbsyn30N?si=13ba22eb0dab44ba', 'https://open.spotify.com/playlist/5kjhVUgYNyu7zZ1rVEgnAm?si=736fffa3ffd74542', 'https://open.spotify.com/playlist/4MJf4EASw6fzzfJlUFv5l0?si=69b9e7f865114349', 'https://open.spotify.com/playlist/0kdd8kYfLWj1f6Ln0clqTI?'],
    ['https://open.spotify.com/playlist/3V2DajyxOv0Nnfk0AwjcxV?si=06c7f948c9444d3d', 'https://open.spotify.com/playlist/2NesB7u53TUuMUTUrkntLL?si=95cd05d369c04c6a', 'https://open.spotify.com/playlist/0ybME665syNkRpRUIRcKqj?si=2a728d1376d448bb', 'https://open.spotify.com/playlist/3763Ou1Xrz42HAwONkjcZh?'],
    ['https://open.spotify.com/playlist/34FE23iG477W2CrtcCw6NM?si=bc0fd78a986a4e03', 'https://open.spotify.com/playlist/5W4HtI15pWZLiALqm0XWhb?si=e6a3ffb7fda44c61', 'https://open.spotify.com/playlist/4OrOYDkvh9bZ0apjdbXnDd?si=c9127ffd33e843d5'],
    ['https://open.spotify.com/playlist/6q8Uqvbgf01UFYK3NjgKOa?si=2f5c2e1a201244d4', 'https://open.spotify.com/playlist/2MQ1o8Fg21lHJjRTrRzyfl?si=8dc19716b3ed4c24', 'https://open.spotify.com/playlist/1UBn0YJoWBgy93m94mxRpC?si=09dd06b02fd94314', 'https://open.spotify.com/playlist/7Co3v9HoxO1tR2FCjsEcjS?'],
    ['https://open.spotify.com/playlist/44V6F2OpcEgCySkClW5G0n?si=be620ba860974eb9', 'https://open.spotify.com/playlist/1b6TtvoM7h8LqF9d9mKYHg?si=5167bfce7ee54cae', 'https://open.spotify.com/playlist/34TkiR7rBPL38HiUhIAja2?si=698396ac02b04a4d'],
    ['https://open.spotify.com/playlist/3PoCb5MYz0AoDNf4iQvTuW?si=10fdb4af683a4d3f', 'https://open.spotify.com/playlist/27ESv3ew8e6x4t0CFADH8W?si=7e11628658164d1e', 'https://open.spotify.com/playlist/3se77cC4MY9PcMZrCqIoQL?si=94b631da2f6041fa'],
]

playlist_count = 1
for i in range(len(playlists)):
    for j in range(len(playlists[i])):
        playlist_count += 1

num = 1
playlist_infos = [ [] for _ in range(12) ]
for i in range(len(playlists)):
    for j in range(len(playlists[i])):
        playlist_uri = get_playlist_uri(playlists[i][j])

        print(f"Playlist {num}/{playlist_count}")
        playlist_infos[i].append(get_track_ids_and_store_audio_features(access_token, playlist_uri))
        num += 1

with open('data/playlist_data.json', 'w', encoding='utf-8') as f:
    json.dump({ 'data': playlist_infos }, f, ensure_ascii=False, indent=4)


Playlist 1/44
fetching playlist 7BIFvKEvSWfr0c1Ur2PxQ5
0
Playlist 2/44
fetching playlist 6w2nktImGPS83JiAkG2VzB
0
1
2
3
4
5
6
Playlist 3/44
fetching playlist 5NZj30WhMY4gnMIGPD8gir
0
1
Playlist 4/44
fetching playlist 7LDJsFK7uk53RNxax3z2YC
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Playlist 5/44
fetching playlist 0eN9zu6ZezHsNodVtHIlAU
0
1
2
3
4
Playlist 6/44
fetching playlist 6unGtxWr0zGP2GJQijrzTW
0
1
2
3
4
5
6
7
Playlist 7/44
fetching playlist 3kTgfkfU6GomvrEyNgC7LV
0
Playlist 8/44
fetching playlist 6SP3BZKRwpp5PX0rI1Rmem
0
1
2
3
4
5
6
Playlist 9/44
fetching playlist 1AmBxyen2AaSKf6xgl4FAa
0
1
2
3
4
5
6
7
8
Playlist 10/44
fetching playlist 6zut6b4UaLmsxVPvTHAyUC
0
1
Playlist 11/44
fetching playlist 3n5J91l343pfMflirtBZhD
0
1
2
Playlist 12/44
fetching playlist 5Wlx45esaqtfimuHPGYaF4
0
1
Playlist 13/44
fetching playlist 003iGZIa5Ne43VBOjxe8OQ
0
Playlist 14/44
fetching playlist 3GzH2VIpDXQaouhWJeBvYn
0
Playlist 15/44
fetching playlist 4OUEfwVUeRHzAKW5rGjH3M
0
Playlist 16/44
fetching playlis

In [27]:
zodiacValues = [['Aries', '♈'], ['Taurus', '♉'], ['Gemini', '♊'], ['Cancer', '♋'], ['Leo', '♌'], ['Virgo', '♍'], ['Libra', '♎'], ['Scorpio', '♏'], ['Sagittarius', '♐'], ['Capricorn', '♑'], ['Aquarius', '♒'], ['Pisces', '♓']]

with open('data/playlist_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

    for i in range(len(data['data'])):
        track_count = 0
        for j in range(len(data['data'][i])):
            track_count += len(data['data'][i][j]['track_ids'])

        print(f"{i + 1} - {zodiacValues[i][0]}: {track_count} songs")

1 - Aries: 409 songs
2 - Taurus: 1350 songs
3 - Gemini: 755 songs
4 - Cancer: 332 songs
5 - Leo: 312 songs
6 - Virgo: 510 songs
7 - Libra: 363 songs
8 - Scorpio: 431 songs
9 - Sagittarius: 476 songs
10 - Capricorn: 427 songs
11 - Aquarius: 433 songs
12 - Pisces: 592 songs


In [264]:
tracks_table = dynamodb.Table('SPOTIFY_TRACKS')
json_to_write = {'data': [ [] for _ in range(12) ]}

def Decimal(number):
    if float(number)%1==0:
        return int(number)
    return float(number)

with open('data/playlist_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
    for i in range(len(data['data'])):
        zodiac = data['data'][i]


        for j in range(len(zodiac)):
            json_to_write['data'][i].append([])
            
            track_ids = zodiac[j]['track_ids']
            track_ids = list(dict.fromkeys(track_ids))
            map_track_ids = []
            for track_id in track_ids:
                map_track_ids.append({'id': track_id})

            index = 0
            while index < len(track_ids) - 1:
                db_response = dynamodb.batch_get_item(
                    RequestItems={
                        'SPOTIFY_TRACKS': {
                            'Keys': map_track_ids[index:index + 100]
                        }
                    }
                )

                track_received = db_response['Responses']['SPOTIFY_TRACKS']
                for track in track_received:
                    data_to_save = {}
                    properties = ['valence', 'danceability', 'instrumentalness', 'acousticness', 'liveness', 'energy', 'speechiness', 'tempo', 'key', 'loudness']

                    for prop in properties:
                        data_to_save[prop] = float(track[prop])
                    
                    json_to_write['data'][i][j].append(data_to_save)


                index += len(track_received) - 1

    with open('data/raw_playlist_data.json', 'w', encoding='utf-8') as f:
        json.dump(json_to_write, f, ensure_ascii=False, indent=4)

In [24]:
value_to_look = ['valence', 'danceability', 'instrumentalness', 'acousticness', 'liveness', 'energy', 'speechiness', 'tempo', 'loudness']

table = dynamodb.Table('SPOTIFY_TRACKS')

response = table.scan()
data = response['Items']

while 'LastEvaluatedKey' in response:
    response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

print(f"there are {len(data)} entries")

max_items = []
min_items = []
for value in value_to_look:
    max_items.append({value: -999.9})
    min_items.append({value: 999.9})
for item in data:
    for i in range(len(value_to_look)):
        value = value_to_look[i]

        item[value] = float(item[value])
        if item[value] > max_items[i][value]:
            max_items[i] = item
        if item[value] < max_items[i][value]:
            min_items[i] = item

for i in range(len(value_to_look)):
    value = value_to_look[i]

    print(f"max {value} - {max_items[i]['id']} ({max_items[i][value]})")
    print(f"min {value} - {min_items[i]['id']} ({min_items[i][value]})")


there are 9338 entries
max valence - 56KyV36puztkiJ62ca3D1t (1.0)
min valence - 3Wrjm47oTz2sjIgck11l5e (0.589)
max danceability - 03qvygTWSmCQsoiBnIJvrc (0.985)
min danceability - 3Wrjm47oTz2sjIgck11l5e (0.714)
max instrumentalness - 6mXywNQnfuVBttZuzQYUKB (0.992)
min instrumentalness - 3Wrjm47oTz2sjIgck11l5e (0.0)
max acousticness - 3OvCTtPuo6ZkOWxp23mUd0 (0.996)
min acousticness - 3Wrjm47oTz2sjIgck11l5e (0.127)
max liveness - 7lpC8rp4pAQTADY7b8ezE1 (0.998)
min liveness - 3Wrjm47oTz2sjIgck11l5e (0.359)
max energy - 7lOvJBqH38ZY59eLU7bioq (0.999)
min energy - 3Wrjm47oTz2sjIgck11l5e (0.8)
max speechiness - 72jcrsvDnZt5dpWKzku8MO (0.944)
min speechiness - 3Wrjm47oTz2sjIgck11l5e (0.0504)
max tempo - 0BfRuvONY7dhBcTLO9I4NK (239.657)
min tempo - 3Wrjm47oTz2sjIgck11l5e (134.002)
max loudness - 1PvottCnfZi3RCZVVZ2WtD (4.638)
min loudness - 3Wrjm47oTz2sjIgck11l5e (-4.808)


# Profile Playlist Scraper

In [72]:
# get Spotify's playlists:
playlists = []

def get_playlists_from_user(user: str):
    i = 0
    next_url = f"https://api.spotify.com/v1/users/{user}/playlists?limit=20"
    while next_url != None:
        headers = {
            'Authorization': f"Bearer {access_token}"
        }

        response = requests.get(url = next_url, headers = headers)

        if response.status_code == 200:
            raw_json = response.json()

            for playlist in raw_json['items']:
                playlists.append({
                    'id': playlist['id'],
                    'name': playlist['name']
                })
                
                i += 1
                print(f"{i}/{raw_json['total']}")

            next_url = raw_json['next']
        else:
            print(response.text)

get_playlists_from_user('andrew03330')
get_playlists_from_user('spotify')

json_to_write = {'data': playlists}
with open('data/spotify_playlists.json', 'w', encoding='utf-8') as f:
    json.dump(json_to_write, f, ensure_ascii=False, indent=4)

1/122
2/122
3/122
4/122
5/122
6/122
7/122
8/122
9/122
10/122
11/122
12/122
13/122
14/122
15/122
16/122
17/122
18/122
19/122
20/122
21/122
22/122
23/122
24/122
25/122
26/122
27/122
28/122
29/122
30/122
31/122
32/122
33/122
34/122
35/122
36/122
37/122
38/122
39/122
40/122
41/122
42/122
43/122
44/122
45/122
46/122
47/122
48/122
49/122
50/122
51/122
52/122
53/122
54/122
55/122
56/122
57/122
58/122
59/122
60/122
61/122
62/122
63/122
64/122
65/122
66/122
67/122
68/122
69/122
70/122
71/122
72/122
73/122
74/122
75/122
76/122
77/122
78/122
79/122
80/122
81/122
82/122
83/122
84/122
85/122
86/122
87/122
88/122
89/122
90/122
91/122
92/122
93/122
94/122
95/122
96/122
97/122
98/122
99/122
100/122
101/122
102/122
103/122
104/122
105/122
106/122
107/122
108/122
109/122
110/122
111/122
112/122
113/122
114/122
115/122
116/122
117/122
118/122
119/122
120/122
121/122
122/122


KeyboardInterrupt: 

In [74]:
with open('data/spotify_playlists.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

    length = len(data['data'])
    for i in range(1074, length):
        id = data['data'][i]['id']
        name = data['data'][i]['name']

        print(f"{i + 1}/{length} ({name})")
        get_track_ids_and_store_audio_features(access_token, id)

1075/1224 (Girls' Night)
fetching playlist 37i9dQZF1DX0Uv9tZ47pWo
batch 1
batch 2
1076/1224 (Summer House 2024)
fetching playlist 37i9dQZF1DX05r4Oy3Ln97
batch 1
{
  "error" : {
    "status" : 429
  }
}
batch 2


KeyboardInterrupt: 

In [67]:
playlist_ids = ['6yPiKpy7evrwvZodByKvM9', '5S8SJdl1BDc0ugpkEvFsIL', '0z085jfTrSXxhFUyKKExxJ', '7I6XVNQVayB46Zzh4GNPoU']

for id in playlist_ids:
    get_track_ids_and_store_audio_features(access_token, id)

fetching playlist 6yPiKpy7evrwvZodByKvM9
batch 1
{
  "error" : {
    "status" : 429
  }
}
batch 2
{
  "error" : {
    "status" : 429
  }
}
batch 3


KeyboardInterrupt: 